In [1]:
import os

while os.getcwd() != "/" and ".gitignore" not in os.listdir(os.getcwd()):
	os.chdir("..")
	if os.getcwd() == "/":
		print("COULD NOT FIND gitignore.  Invalid project base file.")
print("Current Working Directory:  ", os.getcwd())

Current Working Directory:   /home/camer/capstone/numaware


In [2]:
from joint_model.src.uafscs.utils import data_utils		as dutils
from joint_model.src.uafscs.utils import train_utils    as tutils

import os
import torch.nn as nn
import torch

In [3]:
dir = "/home/camer/capstone/numaware/data/data"
imgs = os.path.join(dir,"images")
test_data = os.path.join(dir,"test_data.csv")
train_data = os.path.join(dir,"train_data.csv")

In [4]:
train_data

'/home/camer/capstone/numaware/data/data/train_data.csv'

In [5]:
test_dataset = dutils.UADataset(imgs,test_data)
test_loader  = torch.utils.data.DataLoader(test_dataset,batch_size=16)

In [6]:
#Thankyou Dr

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Convolution layers with pooling
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)  # First conv layer
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling after the first conv layer
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)  # Second conv layer
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling after the second conv layer
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)  # Third conv layer
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling after the third conv layer
        
        # Flattened size after pooling
        self.flattened_size = 307202  # This is the correct flattened size after all pooling

        # Fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 128)  # Fixed flattened size
        self.fc2 = nn.Linear(128, 3)  # Output 3 floats for regression task

        # Activation functions
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, midpoints):
        # Apply convolution layers with ReLU activation and pooling
        x = self.relu(self.conv1(x))
        x = self.pool1(x)  # Apply max pooling
        
        x = self.relu(self.conv2(x))
        x = self.pool2(x)  # Apply max pooling
        
        x = self.relu(self.conv3(x))
        x = self.pool3(x)  # Apply max pooling
        
       
        x = torch.flatten(x, start_dim=1) # I made this
        x = torch.cat((x, midpoints), 1)  # I made this

        # Fully connected layers with dropout
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)  # Output layer for regression
        
        return x



model = SimpleCNN()


In [7]:
trainer = tutils.UAFSTrainer( model            = model,
                              targets          = "carts",
                              midpoints        = "yolo",
                              lr               = 0.01,
                              epochs           = 2,
                              train_dataloader = test_loader, #Used the test set as training just so it could finish faster
                              test_dataloader  = test_loader,
                              optimizer        = "AdamW",
                              loss_fn          = "MSE" 
                            )

In [8]:
trainer.train()

100%|██████████| 119/119 [02:18<00:00,  1.17s/it]


Epoch: 1   Training Loss: 15372.7393  Training R2 score: -170.1665527699864 


100%|██████████| 119/119 [02:18<00:00,  1.16s/it]

Epoch: 2   Training Loss: 16236.2764  Training R2 score: -29.256300655281063 


In [8]:
trainer.eval()

100%|██████████| 119/119 [01:08<00:00,  1.73it/s]

here
Test Loss: 70652.8125  Test R2 score: -209.66723038448004 
